In [1]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import scale
from tensorflow.keras.layers import Flatten, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow.keras.backend as K
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D, AvgPool2D

In [2]:
import numpy as np

distance = np.load('distance_ILMI_1000+393_random_dtw_matrix.npy')
y_data = np.load('y_data_ILMI_1000+393_random_dtw_matrix.npy')
patient_ids = np.load('patient_ids_ILMI_1000+393_random_dtw_matrix.npy')

In [3]:
loaded_data = np.load('matrices_data.npz')
cross_corr_loaded = loaded_data['cross_corr']
coherence_loaded = loaded_data['coherence']
pli_loaded = loaded_data['pli']
plv_loaded = loaded_data['plv']

ynew = np.load(r"ynew_ALMI_800+164_kernels5000.npy")
patient_ids = np.load(r"patient_ids_ILMI_1000+393.npy")

In [10]:
DTW_exp = np.zeros((6965, 12, 12))
for i in range(distance.shape[0]):
    min_value = np.min(distance[i])
    max_value = np.max(distance[i])

    n_dist = (distance[i] - min_value) / (max_value - min_value)

    DTW_exp[i] = np.exp(- np.abs(n_dist))

In [15]:
DTW_exp.shape

(6965, 12, 12)

In [20]:
def create_K(A_i, B_i, C_i, D_i):
    top_row = np.concatenate((A_i, B_i), axis=1)
    bottom_row = np.concatenate((C_i, D_i), axis=1)
    return np.concatenate((top_row, bottom_row), axis=0)

K_matrices=[]
for i in range(ynew.shape[0]):
    K_i = create_K(cross_corr_loaded[i], coherence_loaded[i], DTW_exp[i], plv_loaded[i])
    K_matrices.append(K_i)

In [21]:
t = np.array(K_matrices)

In [22]:
t.shape

(6965, 24, 24)

In [23]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GroupKFold
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GroupKFold
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.losses import BinaryCrossentropy

X_data = np.stack(t, axis=-1)

gkf = GroupKFold(n_splits=5)

sum_accuracy = 0
sum_sensitivity = 0
sum_specificity = 0

for train_idx, test_idx in gkf.split(t, ynew, groups=patient_ids):
    X_train, X_test = t[train_idx], t[test_idx]
    y_train, y_test = ynew[train_idx], ynew[test_idx]

    tf.keras.backend.clear_session()

    model = Sequential()

    # Use 4 input channels
    model.add(Conv2D(10, (3, 3), input_shape=(24, 24, 1)))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Flatten())

    model.add(Dense(128, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss=BinaryCrossentropy(),
                  optimizer=RMSprop(),
                  metrics=['accuracy'])

    history = model.fit(X_train, y_train,
                        batch_size=16,
                        epochs=20,
                        verbose=1,
                        validation_split=0.25)

    predictions = model.predict(X_test)
    y_pred = np.round(predictions).astype(int).transpose()
    y_pred = y_pred[0]

    y_pred_majority = []
    for patient_idx in np.unique(patient_ids[test_idx]):
        segment_predictions = y_pred[patient_ids[test_idx] == patient_idx]
        majority_vote = np.bincount(segment_predictions).argmax()
        y_pred_majority.append(majority_vote)

    y_test_majority = []
    for patient_idx in np.unique(patient_ids[test_idx]):
        segment_predictions = y_test[patient_ids[test_idx] == patient_idx]
        majority_vote = np.bincount(segment_predictions).argmax()
        y_test_majority.append(majority_vote)

    accuracy = accuracy_score(y_test_majority, y_pred_majority)
    sum_accuracy += accuracy
    # Calculate confusion matrix
    tn, fp, fn, tp = confusion_matrix(y_test_majority, y_pred_majority).ravel()

    # Calculate sensitivity and specificity
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)

    sum_sensitivity += sensitivity
    sum_specificity += specificity

    print(confusion_matrix(y_test_majority, y_pred_majority))
    print("Accuracy:", accuracy)
    print("Sensitivity:", sensitivity)
    print("Specificity:", specificity)

average_accuracy = sum_accuracy / 5
average_sensitivity = sum_sensitivity / 5
average_specificity = sum_specificity / 5

print("Average Accuracy:", average_accuracy)
print("Average Sensitivity:", average_sensitivity)
print("Average Specificity:", average_specificity)


Epoch 1/20
262/262 [==============================] - 1s 3ms/step - loss: 0.3080 - accuracy: 0.8698 - val_loss: 0.2575 - val_accuracy: 0.8973
Epoch 2/20
262/262 [==============================] - 1s 2ms/step - loss: 0.2039 - accuracy: 0.9133 - val_loss: 0.2177 - val_accuracy: 0.9081
Epoch 3/20
262/262 [==============================] - 1s 2ms/step - loss: 0.1682 - accuracy: 0.9303 - val_loss: 0.2076 - val_accuracy: 0.9232
Epoch 4/20
262/262 [==============================] - 1s 2ms/step - loss: 0.1443 - accuracy: 0.9452 - val_loss: 0.3075 - val_accuracy: 0.8973
Epoch 5/20
262/262 [==============================] - 1s 2ms/step - loss: 0.1271 - accuracy: 0.9497 - val_loss: 0.5263 - val_accuracy: 0.8593
Epoch 6/20
262/262 [==============================] - 1s 2ms/step - loss: 0.1148 - accuracy: 0.9559 - val_loss: 0.1929 - val_accuracy: 0.9296
Epoch 7/20
262/262 [==============================] - 1s 2ms/step - loss: 0.0932 - accuracy: 0.9634 - val_loss: 0.1818 - val_accuracy: 0.9332
Epoch 

262/262 [==============================] - 1s 2ms/step - loss: 0.0132 - accuracy: 0.9962 - val_loss: 0.4605 - val_accuracy: 0.9196
Epoch 17/20
262/262 [==============================] - 1s 2ms/step - loss: 0.0134 - accuracy: 0.9952 - val_loss: 0.6789 - val_accuracy: 0.8837
Epoch 18/20
262/262 [==============================] - 1s 2ms/step - loss: 0.0114 - accuracy: 0.9952 - val_loss: 0.5055 - val_accuracy: 0.9261
Epoch 19/20
262/262 [==============================] - 1s 2ms/step - loss: 0.0074 - accuracy: 0.9969 - val_loss: 0.6312 - val_accuracy: 0.9218
Epoch 20/20
44/44 [==============================] - 0s 1ms/step
[[ 79   6]
 [  9 185]]
Accuracy: 0.946236559139785
Sensitivity: 0.9536082474226805
Specificity: 0.9294117647058824
Epoch 1/20
262/262 [==============================] - 1s 2ms/step - loss: 0.3870 - accuracy: 0.8328 - val_loss: 0.3784 - val_accuracy: 0.8494
Epoch 2/20
262/262 [==============================] - 1s 2ms/step - loss: 0.2678 - accuracy: 0.8902 - val_loss: 0.2830

In [32]:
conc = np.zeros((6965,312))
for i in range(distance.shape[0]):
    conc[i] = concatenate_upper_triangular_vectors(cross_corr_loaded[i],coherence_loaded[i],plv_loaded[i],pli_loaded[i])

In [33]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import SGDClassifier
import numpy as np

gkf = GroupKFold(n_splits=5)

sum_accuracy = 0
sum_sensitivity = 0
sum_specificity = 0
sum_f1 = 0

for train_idx, test_idx in gkf.split(conc, ynew, groups=patient_ids):
    X_train, X_test = conc[train_idx], conc[test_idx]
    y_train, y_test = ynew[train_idx], ynew[test_idx]

    classifier = SGDClassifier()
    classifier.fit(X_train, y_train)

    y_pred = classifier.predict(X_test)

    y_pred_majority = []
    for patient_idx in np.unique(patient_ids[test_idx]):
        segment_predictions = y_pred[patient_ids[test_idx] == patient_idx]
        majority_vote = np.bincount(segment_predictions).argmax()
        y_pred_majority.append(majority_vote)

    y_test_majority = []
    for patient_idx in np.unique(patient_ids[test_idx]):
        segment_predictions = y_test[patient_ids[test_idx] == patient_idx]
        majority_vote = np.bincount(segment_predictions).argmax()
        y_test_majority.append(majority_vote)

    accuracy = accuracy_score(y_test_majority, y_pred_majority)
    sum_accuracy += accuracy

    tn, fp, fn, tp = confusion_matrix(y_test_majority, y_pred_majority).ravel()

    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    f1 = f1_score(y_test_majority, y_pred_majority)

    sum_sensitivity += sensitivity
    sum_specificity += specificity


    print(confusion_matrix(y_test_majority, y_pred_majority))
    print("Accuracy:", accuracy)
    print("Sensitivity:", sensitivity)
    print("Specificity:", specificity)


average_accuracy = sum_accuracy / 5
average_sensitivity = sum_sensitivity / 5
average_specificity = sum_specificity / 5


print("Average Accuracy:", average_accuracy)
print("Average Sensitivity:", average_sensitivity)
print("Average Specificity:", average_specificity)


[[ 64  18]
 [  4 193]]
Accuracy: 0.921146953405018
Sensitivity: 0.9796954314720813
Specificity: 0.7804878048780488
[[ 56  22]
 [  2 199]]
Accuracy: 0.9139784946236559
Sensitivity: 0.9900497512437811
Specificity: 0.717948717948718
[[ 78   7]
 [ 13 181]]
Accuracy: 0.9283154121863799
Sensitivity: 0.9329896907216495
Specificity: 0.9176470588235294
[[ 49  25]
 [  2 202]]
Accuracy: 0.9028776978417267
Sensitivity: 0.9901960784313726
Specificity: 0.6621621621621622
[[ 62  12]
 [ 15 189]]
Accuracy: 0.9028776978417267
Sensitivity: 0.9264705882352942
Specificity: 0.8378378378378378
Average Accuracy: 0.9138392511797016
Average Sensitivity: 0.9638803080208358
Average Specificity: 0.7832167163300592


In [ ]:
# # for 4 channel
# t = []
# for i in range(ynew.shape[0]):
#     t2 = []
#     t2.append(cross_corr_loaded[i])
#     t2.append(pli_loaded[i])
#     t2.append(plv_loaded[i])
#     t2.append(coherence_loaded[i])
#     t.append(np.array(t2).reshape(12,12,4))
# t = np.array(t)

In [4]:
def extract_upper_triangular(matrix):
    n = len(matrix)
    upper_triangular_array = []

    for i in range(n):
        for j in range(i, n):
            upper_triangular_array.append(matrix[i][j])

    return upper_triangular_array

def concatenate_upper_triangular_vectors(matrix1, matrix2, matrix3, matrix4):
    vector1 = extract_upper_triangular(matrix1)
    vector2 = extract_upper_triangular(matrix2)
    vector3 = extract_upper_triangular(matrix3)
    vector4 = extract_upper_triangular(matrix4)

    concatenated_vector = vector1 + vector2 + vector3 + vector4
    return concatenated_vector

In [ ]:
# t2=[]
# for i in range(distance.shape[0]):
#     K_i = extract_upper_triangular(distance[i])
#     t2.append(K_i)
# t2 = np.array(t2)
# print(t2.shape)